In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [7]:
import csv
import emoji

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Wojciech\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
dataset = []
with open('../data/reddit.csv', newline='',  encoding="utf8") as data_file:
    reader = csv.reader(data_file, delimiter=',', quotechar='\"')
    for row in reader:
        id = row[0]
        # replace emojis with tokens
        side_a_1 = row[1]
        side_b_1 = row[2]
        side_a_2 = row[3]
        # normalize all whitespaces to space
        side_a_1 = ' '.join(side_a_1.split())
        side_b_1 = ' '.join(side_b_1.split())
        side_a_2 = ' '.join(side_a_2.split())

        dataset.append([id, side_a_1, side_b_1, side_a_2])

In [51]:
# remove duplicates
messages = []
for row in dataset:
    if row[1] not in messages:
        messages.append(row[1])
    if row[2] not in messages:
        messages.append(row[2])
    if row[3] not in messages:
        messages.append(row[3])

In [11]:
from nltk import sent_tokenize
from nltk.tokenize import TweetTokenizer

tokenized_sentences = []

tokenizer = TweetTokenizer()
for message in messages:
    sentences = [ [emoji.demojize(token.lower()) for token in tokenizer.tokenize(sentence)] for sentence in sent_tokenize(message)]
    tokenized_sentences.append(sentences)


In [47]:
corpus = []
for sentences in tokenized_sentences:
    corpus += sentences

In [50]:
corpus[94]

['my',
 'mistake',
 'sorry',
 'about',
 'that',
 'no',
 'i',
 "didn't",
 'forget',
 'about',
 'you',
 'at',
 'all',
 ':grinning_face:',
 ':smiling_face_with_smiling_eyes:',
 ':smiling_face:']

In [34]:
import sys
sys.path.insert(0, '../tf_glove')
from tf_glove import GloVeModel

glove_model = GloVeModel(embedding_size=100, context_size=5, min_occurrences=10, learning_rate=0.05, batch_size=512)
glove_model.fit_to_corpus(corpus)

In [35]:
glove_model.train(num_epochs=300, log_dir="../glove_logs", summary_batch_interval=5000)

print('words:', len(glove_model.words))

Writing TensorBoard summaries to ../glove_logs
words: 4052


In [36]:
%matplotlib inline
glove_model.generate_tsne(path="../map.png", word_count=6849)

In [42]:
def prepare_input(text: str):
   tokenizer = TweetTokenizer()
   tokens = [emoji.demojize(token.lower()) for token in tokenizer.tokenize(text)]
   vectors = [glove_model.embedding_for(token) for token in tokens]
   return vectors

In [52]:
embedding = prepare_input("I <3 your birthday party!") 